# MCP Agent

- Requires distributions/masterclass-agents/run.yaml

In [1]:
from llama_stack_client import LlamaStackClient, Agent, AgentEventLogger


In [2]:
client = LlamaStackClient(
    base_url="http://localhost:8321",
)

In [3]:
client.models.list()

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/models "HTTP/1.1 200 OK"


[Model(identifier='all-MiniLM-L6-v2', metadata={'embedding_dimension': 384.0}, api_model_type='embedding', provider_id='ollama', type='model', provider_resource_id='all-minilm:latest', model_type='embedding'),
 Model(identifier='llama3.2:3b-instruct-fp16', metadata={}, api_model_type='llm', provider_id='ollama', type='model', provider_resource_id='llama3.2:3b-instruct-fp16', model_type='llm')]

In [4]:
client.tools.list()

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools "HTTP/1.1 500 Internal Server Error"
INFO:llama_stack_client._base_client:Retrying request to /v1/tools in 0.485506 seconds
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools "HTTP/1.1 500 Internal Server Error"
INFO:llama_stack_client._base_client:Retrying request to /v1/tools in 0.980544 seconds
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools "HTTP/1.1 500 Internal Server Error"


InternalServerError: Error code: 500 - {'detail': 'Internal server error: An unexpected error occurred.'}

In [5]:
client.toolgroups.list()

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/toolgroups "HTTP/1.1 200 OK"


[ToolGroup(identifier='builtin::rag', provider_id='rag-runtime', type='tool_group', args=None, mcp_endpoint=None, provider_resource_id='builtin::rag'),
 ToolGroup(identifier='builtin::websearch', provider_id='tavily-search', type='tool_group', args=None, mcp_endpoint=None, provider_resource_id='builtin::websearch'),
 ToolGroup(identifier='builtin::wolfram_alpha', provider_id='wolfram-alpha', type='tool_group', args=None, mcp_endpoint=None, provider_resource_id='builtin::wolfram_alpha'),
 ToolGroup(identifier='mcp::filesystem', provider_id='model-context-protocol', type='tool_group', args=None, mcp_endpoint=McpEndpoint(uri='http://localhost:8000/sse'), provider_resource_id='mcp::filesystem')]

In [6]:
# alternatively can have this in the run.yaml file
client.toolgroups.register(
    toolgroup_id="mcp::filesystem",
    provider_id="model-context-protocol",
    mcp_endpoint={"uri": "http://localhost:8000/sse"},
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/toolgroups "HTTP/1.1 200 OK"


In [7]:
client.toolgroups.list()

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/toolgroups "HTTP/1.1 200 OK"


[ToolGroup(identifier='builtin::rag', provider_id='rag-runtime', type='tool_group', args=None, mcp_endpoint=None, provider_resource_id='builtin::rag'),
 ToolGroup(identifier='builtin::websearch', provider_id='tavily-search', type='tool_group', args=None, mcp_endpoint=None, provider_resource_id='builtin::websearch'),
 ToolGroup(identifier='builtin::wolfram_alpha', provider_id='wolfram-alpha', type='tool_group', args=None, mcp_endpoint=None, provider_resource_id='builtin::wolfram_alpha'),
 ToolGroup(identifier='mcp::filesystem', provider_id='model-context-protocol', type='tool_group', args=None, mcp_endpoint=McpEndpoint(uri='http://localhost:8000/sse'), provider_resource_id='mcp::filesystem')]

In [10]:
from llama_stack_client.types.shared_params.agent_config import ToolConfig

agent = Agent(
    client,
    model="llama3.2:3b-instruct-fp16",
    instructions="You are a helpful assistant.",
    tools=["mcp::filesystem"],
)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/agents "HTTP/1.1 200 OK"


INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=mcp%3A%3Afilesystem "HTTP/1.1 500 Internal Server Error"
INFO:llama_stack_client._base_client:Retrying request to /v1/tools in 0.496002 seconds
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=mcp%3A%3Afilesystem "HTTP/1.1 500 Internal Server Error"
INFO:llama_stack_client._base_client:Retrying request to /v1/tools in 0.822238 seconds
INFO:httpx:HTTP Request: GET http://localhost:8321/v1/tools?toolgroup_id=mcp%3A%3Afilesystem "HTTP/1.1 500 Internal Server Error"


InternalServerError: Error code: 500 - {'detail': 'Internal server error: An unexpected error occurred.'}

In [11]:
user_prompts = [
    "Hello",
    "do you have access to any tools?",
    "list the directories that you have access to",
]

session_id = agent.create_session("test-session")
for prompt in user_prompts:
    print(f"User> {prompt}")
    response = agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
    )

    for log in AgentEventLogger().log(response):
        log.print()

NameError: name 'agent' is not defined